In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd

In [2]:
main_link = 'https://eda.yandex/shop/azbuka_vkusa_simferopolskij_bulvar_22k3?category=782350'
response = requests.get(main_link).text
soup = bs(response,'lxml')


In [3]:
eda_block = soup.find('div',{'class':'RestaurantPageStorePageCategoryListWithItems_root'})
eda_list = eda_block.find_all('div',{'class':'RestaurantStorePageMenuCategory_itemWrapper'})
pprint(eda_list)

[<div class="RestaurantStorePageMenuCategory_itemWrapper"><div class="RestaurantStorePageMenuItem_root RestaurantStorePageMenuItem_isPharmacy"><div></div><div class="RestaurantStorePageMenuItem_topLine"><div class="RestaurantStorePageMenuItem_priceWrapper"><div class="UILoader_root"><div class="UILoader_content"><div class="RestaurantStorePageMenuItem_priceAndCountWrapper"><span class="RestaurantStorePageMenuItem_price">139<!-- --> <!-- -->₽</span></div></div><div class="UILoader_backdrop"></div></div></div><h3 class="RestaurantStorePageMenuItem_title">Крупа гречневая Образ Жизни Алтая зеленая 5х70г в пакетиках</h3> <span class="RestaurantStorePageMenuItem_weight">350 г</span></div><div class="RestaurantStorePageMenuItem_pictureContainer"><div class="UIMagicalImage_root RestaurantStorePageMenuItem_picture"><div aria-label="Крупа гречневая Образ Жизни Алтая зеленая 5х70г в пакетиках" class="UIMagicalImage_image RestaurantStorePageMenuItem_pictureImage" role="img"></div><noscript><img al

In [92]:
bakalea = []
for eda in eda_list:
    eda_data = {}
    eda_name = eda.find('h3').getText()
    eda_weight = eda.find('span',{'class':'RestaurantStorePageMenuItem_weight'}).getText()
    eda_price =  eda.find('span',{'class':'RestaurantStorePageMenuItem_price'})
  
    if not eda_price:
        eda_price = 0
        eda_curr = None
    else:
        eda_price = eda_price.getText()
        eda_curr = eda_price[-1]
        eda_price = eda_price.replace(' ₽', '')
        
    eda_data['name'] = eda_name
    eda_data['weight'] = eda_weight      
    eda_data['price'] = int(eda_price)
    eda_data['currency'] = eda_curr
    bakalea.append(eda_data)

pprint(bakalea)


[{'currency': '₽',
  'name': 'Крупа гречневая Образ Жизни Алтая зеленая 5х70г в пакетиках',
  'price': 139,
  'weight': '350 г'},
 {'currency': '₽',
  'name': 'Ризотто Casale Paradiso с белыми грибами (без глютена)',
  'price': 382,
  'weight': '300 г'},
 {'currency': '₽',
  'name': 'Кускус среднего помола  Tipiak',
  'price': 275,
  'weight': '500 г'},
 {'currency': '₽',
  'name': 'Крупа кукурузная Casale Paradiso Полента с белыми грибами',
  'price': 395,
  'weight': '300 г'},
 {'currency': '₽',
  'name': 'Крупа пшеничная булгур, Tipiak',
  'price': 410,
  'weight': '500 г'},
 {'currency': '₽',
  'name': 'Рис басмати, «Просто Азбука», 5х100 г, в пакетиках',
  'price': 165,
  'weight': '500 г'},
 {'currency': '₽',
  'name': 'Рис жасмин, «Просто Азбука», 5х100 г, в пакетиках',
  'price': 178,
  'weight': '500 г'},
 {'currency': '₽',
  'name': 'Рис «Акватика» дикий, «Мистраль»',
  'price': 724,
  'weight': '500 г'},
 {'currency': '₽',
  'name': 'Кускус «Премиум», «Мистраль»',
  'price':

# Это была работа над ошибками
 Вес еды - использован метод find_all. Здесь можно и обычным find найти - вернется сразу тег, а не список. И да, кстати из тега выдергиваем значение текста, чтобы сразу были чистые данные:

eda_weight = eda.find('span',{'class':'RestaurantStorePageMenuItem_weight'}).getText()
С ценой - та же ситуация - забыли getText()

eda_price = eda.find('span',{'class':'RestaurantStorePageMenuItem_price'}).getText()
Раз уж работем с ценой. То в идеале из полученных данных извлечь саму цифру и преобразовать к int - потом очень сильно пригодится.
Кстати также с весом - там вообще можно разбить на две составляющие на саму фийру и на ед. измерения. - тоже будет полезно.

Попрошу вас доделать именно до такого состояния. Поля которые должны присутствовать:
Имя, вес (число int), единицы измерения, цена(число int или float) и в идеале еще добавить магазин откуда собраны данные.
Мы с жтим наором будем работать в следующем ДЗ, поэтому пригодится) Пока оценка "хорошо"

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [51]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [138]:
from pymongo import MongoClient

In [139]:
client = MongoClient('localhost', 27017)

In [140]:
db = client['eda']
collection = db['col']

In [141]:
for i in bakalea:
    try:
        db.collection.insert_one(i)
    except:
        pass

In [143]:
 for  db.collection in  db.collection.find({'price':{'$gt':500}}):
     pprint( db.collection)

{'_id': ObjectId('5efe6b5ed4be4a63f6ea90ac'),
 'currency': '₽',
 'name': 'Рис «Акватика» дикий, «Мистраль»',
 'price': 724,
 'weight': '500 г'}
{'_id': ObjectId('5efe6b5ed4be4a63f6ea90c3'),
 'currency': '₽',
 'name': 'Рис басмати Elonga длиннозерный, Dunar, 1 кг',
 'price': 586,
 'weight': '1000 г'}
{'_id': ObjectId('5efe6b5ed4be4a63f6ea90ca'),
 'currency': '₽',
 'name': 'Рис черный Био odri rice',
 'price': 580,
 'weight': '500 г'}
{'_id': ObjectId('5efe6b5ed4be4a63f6ea90d7'),
 'currency': '₽',
 'name': 'Рис «Арборио» Био, Alce Nero',
 'price': 558,
 'weight': '500 г'}
{'_id': ObjectId('5efe6b5ed4be4a63f6ea90d8'),
 'currency': '₽',
 'name': 'Рис Baldo коричневый нешлифованный Био, Alce Nero',
 'price': 521,
 'weight': '500 г'}
{'_id': ObjectId('5efe6b5ed4be4a63f6ea90d9'),
 'currency': '₽',
 'name': 'Рис Dunar Басмати Elonga Sela пропаренный длиннозерный 1 кг банка',
 'price': 539,
 'weight': '1000 г'}
{'_id': ObjectId('5efe6b5ed4be4a63f6ea90e2'),
 'currency': '₽',
 'name': 'Рис Ermes 